# Projet MonteCarlo 
## Application des méthodes MonteCarlo sur le jeu de Puissance 4

**Auteurs:** Jihed BHAR - Yassine BEN ABDALLAH - Nadia BEN YOUSSEF


----

### Importation des librairies

In [1]:
import numpy as np
import math
import random
from copy import deepcopy

### Implémentation de la classe Connect4Game

Cette classe définit les règles du jeu et gère l'état du plateau

In [ ]:
class Connect4Game:
    def __init__(self):
        self.rows = 6
        self.columns = 7
        self.board = np.zeros((self.rows, self.columns), dtype=int)
        self.current_player = 1
        self.game_over = False
        self.winner = None

    def get_valid_moves(self):
        return [col for col in range(self.columns) if self.board[0][col] == 0]

    def make_move(self, column):
        if column not in self.get_valid_moves():
            return False
        for row in range(self.rows-1, -1, -1):
            if self.board[row][column] == 0:
                self.board[row][column] = self.current_player
                break
        if self.check_win(self.current_player):
            self.game_over = True
            self.winner = self.current_player
        elif not self.get_valid_moves():
            self.game_over = True
        self.current_player = 3 - self.current_player
        return True

    def check_win(self, player):
        for row in range(self.rows):
            for col in range(self.columns - 3):
                if all(self.board[row][col+i] == player for i in range(4)):
                    return True
        for row in range(self.rows - 3):
            for col in range(self.columns):
                if all(self.board[row+i][col] == player for i in range(4)):
                    return True
        for row in range(self.rows - 3):
            for col in range(self.columns - 3):
                if all(self.board[row+i][col+i] == player for i in range(4)):
                    return True
                if all(self.board[row+3-i][col+i] == player for i in range(4)):
                    return True
        return False

    def print_board(self):
        print('-' * (2 * self.columns + 1))
        for row in range(self.rows):
            print('|', end='')
            for col in range(self.columns):
                print(' ' if self.board[row][col] == 0 else 'R' if self.board[row][col] == 1 else 'Y', end='|')
            print()
        print('-' * (2 * self.columns + 1))
        print(' ', end='')
        for col in range(self.columns):
            print(f'{col}', end=' ')
        print()

### Implémentation de la classe Node

##### Implémentation de la classe Node pour l'arbre de recherche
Cette classe représente un nœud dans l'arbre MCTS

In [3]:
class Node:
    def __init__(self, game_state, parent=None, move=None):
        self.game_state = deepcopy(game_state)
        self.parent = parent
        self.move = move
        self.children = []
        self.wins = 0
        self.visits = 0
        self.sum_of_squares = 0
        self.untried_moves = game_state.get_valid_moves()

    def select_child(self):
        c = 1.41
        best_score = -float('inf')
        best_child = None
        for child in self.children:
            exploitation = child.wins / child.visits if child.visits > 0 else 0
            exploration = math.sqrt(2 * math.log(self.visits) / child.visits) if child.visits > 0 else float('inf')
            score = exploitation + c * exploration
            if score > best_score:
                best_score = score
                best_child = child
        return best_child

    def add_child(self, move, game_state):
        child = Node(game_state, self, move)
        self.untried_moves.remove(move)
        self.children.append(child)
        return child

    def update(self, result):
        self.visits += 1
        self.wins += result
        self.sum_of_squares += result * result

### Implémentation de l'algorithme Monte Carlo Tree Search (MCTS) standard

Utilise la formule UCB1 standard pour la sélection des nœuds

In [4]:
class MCTS:
    def __init__(self, iterations=1000):  # Réduit à 1000 pour test
        self.iterations = iterations

    def get_best_move(self, game_state):
        root = Node(game_state)
        for _ in range(self.iterations):
            node = root
            state = deepcopy(game_state)
            while node.untried_moves == [] and node.children != []:
                node = node.select_child()
                state.make_move(node.move)
            if node.untried_moves:
                move = random.choice(node.untried_moves)
                state.make_move(move)
                node = node.add_child(move, state)
            while not state.game_over and state.get_valid_moves():
                state.make_move(random.choice(state.get_valid_moves()))  # Simulation aléatoire
            while node:
                result = 0.5 if state.winner is None else 1.0 if state.winner == game_state.current_player else 0.0
                node.update(result)
                node = node.parent
        best_child = max(root.children, key=lambda c: c.visits, default=None)
        return best_child.move if best_child else random.choice(game_state.get_valid_moves())

### Implémentation de l'algorithme UCB1-Tuned

Améliore MCTS en intégrant la variance des récompenses dans la formule de sélection


In [9]:
class UCB1TunedSearch:
    def __init__(self, iterations=1000):  # Réduit à 1000 pour test
        self.iterations = iterations

    def get_best_move(self, game_state):
        root = Node(game_state)
        for _ in range(self.iterations):
            node = root
            state = deepcopy(game_state)
            while node.untried_moves == [] and node.children != []:
                node = self._select_child_ucb_tuned(node)
                state.make_move(node.move)
            if node.untried_moves:
                move = random.choice(node.untried_moves)
                state.make_move(move)
                node = node.add_child(move, state)
            while not state.game_over and state.get_valid_moves():
                state.make_move(random.choice(state.get_valid_moves()))  # Simulation aléatoire
            while node:
                result = 0.5 if state.winner is None else 1.0 if state.winner == game_state.current_player else 0.0
                node.update(result)
                node = node.parent
        best_child = max(root.children, key=lambda c: c.visits, default=None)
        return best_child.move if best_child else random.choice(game_state.get_valid_moves())

    def _select_child_ucb_tuned(self, node):
        log_visits = math.log(node.visits) if node.visits > 0 else 0
        best_score = -float('inf')
        best_child = None
        for child in node.children:
            if child.visits == 0:
                return child
            mean = child.wins / child.visits
            variance = (child.sum_of_squares / child.visits) - (mean * mean) if child.visits > 1 else 0
            variance = min(0.25, max(0, variance))
            two_log_N_over_n = 2 * log_visits / child.visits if child.visits > 0 else float('inf')
            sqrt_two_log_N_over_n = math.sqrt(two_log_N_over_n)
            exploration = math.sqrt(two_log_N_over_n * (variance + sqrt_two_log_N_over_n))
            score = mean + 1.41 * exploration
            if score > best_score:
                best_score = score
                best_child = child
        return best_child


### Fonction de simulation pour comparer les algorithmes

In [10]:
def run_simulation(num_games=100):
    wins_mcts = 0
    wins_ucb_tuned = 0
    draws = 0
    wins_starting = 0
    wins_second = 0

    print(f"Simulation de {num_games} parties entre MCTS et UCB1-Tuned...")
    mcts = MCTS(iterations=1000)
    ucb_tuned = UCB1TunedSearch(iterations=1000)

    for game_num in range(num_games):
        game = Connect4Game()
        AI_red = ucb_tuned if game_num % 2 == 1 else mcts
        AI_yellow = mcts if game_num % 2 == 1 else ucb_tuned
        red_is_mcts = game_num % 2 == 0

        print(f"\nPartie {game_num + 1}/{num_games}")
        print(f"Rouge (commence): {'MCTS' if red_is_mcts else 'UCB1-Tuned'}")
        print(f"Jaune (second): {'UCB1-Tuned' if red_is_mcts else 'MCTS'}")

        last_move = None
        while not game.game_over:
            ai_move = AI_red.get_best_move(game) if game.current_player == 1 else AI_yellow.get_best_move(game)
            game.make_move(ai_move)
            last_move = ai_move

        game.print_board()
        if game.winner is None:
            draws += 1
            print("Résultat: Match nul")
        elif game.winner == 1:
            wins_starting += 1
            if red_is_mcts:
                wins_mcts += 1
                print(f"Résultat: Rouge gagne (MCTS) - Dernier coup: {last_move}")
            else:
                wins_ucb_tuned += 1
                print(f"Résultat: Rouge gagne (UCB1-Tuned) - Dernier coup: {last_move}")
        else:
            wins_second += 1
            if red_is_mcts:
                wins_ucb_tuned += 1
                print(f"Résultat: Jaune gagne (UCB1-Tuned) - Dernier coup: {last_move}")
            else:
                wins_mcts += 1
                print(f"Résultat: Jaune gagne (MCTS) - Dernier coup: {last_move}")

    print("\n--- Statistiques après {} parties ---".format(num_games))
    print(f"MCTS: {wins_mcts} victoires ({wins_mcts/num_games*100:.1f}%)")
    print(f"UCB1-Tuned: {wins_ucb_tuned} victoires ({wins_ucb_tuned/num_games*100:.1f}%)")
    print(f"Matchs nuls: {draws} ({draws/num_games*100:.1f}%)")
    print(f"Rouge (premier): {wins_starting} victoires ({wins_starting/num_games*100:.1f}%)")
    print(f"Jaune (second): {wins_second} victoires ({wins_second/num_games*100:.1f}%)")

### Exécution de la simulation et affichage des résultats

In [ ]:
if __name__ == "__main__":
    run_simulation(100)

Simulation de 100 parties entre MCTS et UCB1-Tuned...

Partie 1/100
Rouge (commence): MCTS
Jaune (second): UCB1-Tuned
---------------
| | | |R|Y| | |
| | |Y|Y|R|R| |
|Y| |R|Y|R|Y| |
|R|Y|R|R|Y|Y| |
|Y|R|Y|R|Y|R| |
|R|R|R|Y|Y|R|Y|
---------------
 0 1 2 3 4 5 6 
Résultat: Jaune gagne (UCB1-Tuned) - Dernier coup: 1

Partie 2/100
Rouge (commence): UCB1-Tuned
Jaune (second): MCTS
---------------
| |R| | | |Y| |
| |Y| |Y|R|R| |
|Y|Y| |R|R|R| |
|Y|Y| |R|R|R| |
|Y|R| |Y|Y|Y| |
|Y|R| |Y|R|R| |
---------------
 0 1 2 3 4 5 6 
Résultat: Jaune gagne (MCTS) - Dernier coup: 0

Partie 3/100
Rouge (commence): MCTS
Jaune (second): UCB1-Tuned
---------------
| | | | | | | |
| | | | | | | |
| | | |R|Y| | |
| | | |Y|R| | |
| | |Y|R|Y| | |
| |Y|R|R|Y| |R|
---------------
 0 1 2 3 4 5 6 
Résultat: Jaune gagne (UCB1-Tuned) - Dernier coup: 4

Partie 4/100
Rouge (commence): UCB1-Tuned
Jaune (second): MCTS
---------------
| | | | | | | |
| |Y| |R| | | |
| |R| |Y| |Y| |
| |R|R|R|R|R| |
| |R|Y|R|Y|Y| |
| |Y|R|R|